# Tool Call LLMs
Some of the more exciting use cases for RC is when you want to cede control of the decision making process to the LLM. RC has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [2]:
from typing import Dict, Any
from typing_extensions import Self

import railtownai_rc as rc
from railtownai_rc.llm import ModelBase, Tool

from src.railtownai_rc.llm import Parameter

In [ ]:
class HarshCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Harsh Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Harsh Critic",
            detail="A tool used to critique a statement harshly.",
            parameters={Parameter("analysis_detail", "The thing you would like to analyze", "string")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
        
    

In [ ]:
class PositiveCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Positive Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Positive Critic",
            detail="A tool used to critique a statement positively.",
            parameters={Parameter("analysis_detail", "The thing you would like to analyze", "string")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)

In [ ]:
class DeeperMeaningCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a critic of the world and you should analyze the given statement and provide a critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Deeper Meaning Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Deeper Meaning Critic",
            detail="A tool used to critique a statement.",
            parameters={Parameter("analysis_detail", "The thing you would like to analyze", "string")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
    
    